In [2]:
import os
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import albumentations as A

# 输入输出路径（保持一致）
input_dir = Path("autodl-tmp/afo/images1")
output_base = Path("autodl-tmp/afo_fog/images1")
output_base.mkdir(parents=True, exist_ok=True)

# ✅ 方法1：简单融合雾
def simple_fog(img, fog_strength=0.4, fog_color=(200, 200, 200)):
    fog_layer = np.ones_like(img, dtype=np.uint8) * np.array(fog_color, dtype=np.uint8)
    foggy = cv2.addWeighted(img, 1 - fog_strength, fog_layer, fog_strength, 0)
    return foggy

# ✅ 方法2：Albumentations 雾增强（伪真实）
albumentations_fogger = A.Compose([
    A.RandomFog(fog_coef_lower=0.4, fog_coef_upper=0.6, alpha_coef=0.1, p=1.0)
])

# ✅ 方法3：Perlin-like 非均匀雾
def perlin_like_fog(img, strength=0.6):
    h, w = img.shape[:2]
    noise = np.random.randn(h, w)
    noise = cv2.GaussianBlur(noise, (0, 0), sigmaX=40)
    fog_mask = cv2.normalize(noise, None, 0, 1, cv2.NORM_MINMAX)
    fog_factor = np.clip(1 - fog_mask * strength, 0, 1)
    fog_color = np.ones_like(img, dtype=np.float32) * 200
    foggy = img * fog_factor[:, :, None] + fog_color * (1 - fog_factor[:, :, None])
    return np.clip(foggy, 0, 255).astype(np.uint8)

# 主循环处理图像
img_list = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png'))])
with tqdm(total=len(img_list), desc="批量生成雾图") as pbar:
    for fname in img_list:
        img_path = input_dir / fname
        img = cv2.imread(str(img_path))

        fog1 = simple_fog(img, fog_strength=0.5)
        fog2 = albumentations_fogger(image=img)["image"]
        fog3 = perlin_like_fog(img, strength=0.8)

        # 保存不同方法生成的图像
        cv2.imwrite(str(output_base / f"simple_{fname}"), fog1)
        cv2.imwrite(str(output_base / f"albumentations_{fname}"), fog2)
        cv2.imwrite(str(output_base / f"perlin_{fname}"), fog3)

        pbar.update(1)


/root/miniconda3/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
批量生成雾图: 100%|██████████| 10/10 [01:27<00:00,  8.80s/it]
